In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
import numpy as np
from datetime import datetime
import os
from pyproj import Transformer

In [41]:
Data = pd.read_csv('Data/AQ_CS_All_Full.csv')
CSMeta = pd.read_csv('Data/CongestionScoot/Metadata(siteCoordinates.csv')
# read the weather data
Weather = pd.read_csv('Data/weather_cleaned.csv')

In [34]:
# initial a transformer to convert the coordinates
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

CSMeta['Longitude'], CSMeta['Latitude'] = zip(*CSMeta.apply(lambda x: transformer.transform(x['Easting'], x['Northing']), axis=1))

In [ ]:


#Merging all CS data and saving it as GiantData.csv

all_CS = CSMeta['ID'].tolist()
#初始化GiantData
GiantData = pd.DataFrame()
# 用for循环来读取"Data/CongestionScoot/MergedCSVs"文件夹下的所有文件名存在于"all_CS"这个list下的csv文件
for i in all_CS:
    filenameurl = f"Data/CongestionScoot/MergedCSVs/{i}.csv"
    if os.path.exists(filenameurl):
        df = pd.read_csv(filenameurl)
        # 'Date'列的格式是17-Dec-2017，'Time'列的格式是0:00，需要合并成一个datetime格式的列
        df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        #将‘DateTime’列的格式设置为17-12-2017 00:00，
        df['DateTime'] = df['DateTime'].dt.strftime('%d-%m-%Y %H:%M')
        #然后选取所有'Time'列的值为xx:00的行
        sub_df = df[df['Time'].str.contains(':00')]
        sub_df['Latitude'] = CSMeta[CSMeta['ID'] == i]['Latitude'].values[0]
        sub_df['Longitude'] = CSMeta[CSMeta['ID'] == i]['Longitude'].values[0]
        #将sub_df的添加到GiantData中
        GiantData = pd.concat([GiantData, sub_df], ignore_index=True)
    else:
        print(f"File {filenameurl} does not exist.")
GiantData.to_csv('Data/CongestionScoot/GiantData.csv', index=False)


In [39]:
GiantData.sample(5)

,DateTime,Date,Time,SatMean,SatBand,FlowMean,Latitude,Longitude
97411,24-01-2018 21:00,24-Jan-2018,21:00,0.00,0-79%,0,51.515349,-0.079868
6553716,19-02-2018 04:00,19-Feb-2018,04:00,4.50,0-79%,87,51.384768,-0.189735
2089583,20-12-2017 22:00,20-Dec-2017,22:00,28.00,0-79%,1332,51.517252,-0.029772
8145113,13-02-2018 04:00,13-Feb-2018,04:00,3.75,0-79%,303,51.532266,-0.233344
6934314,14-03-2018 21:00,14-Mar-2018,21:00,32.50,0-79%,1517,51.412228,-0.300574


In [87]:
GiantData = pd.read_csv('Data/CongestionScoot/GiantData.csv')
GiantData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8990471 entries, 0 to 8990470
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   DateTime   object 
 1   Date       object 
 2   Time       object 
 3   SatMean    float64
 4   SatBand    object 
 5   FlowMean   int64  
 6   Latitude   float64
 7   Longitude  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 548.7+ MB


In [88]:
print(Weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   DateTime                  2184 non-null   datetime64[ns]
 1   temperature_2m (°C)       2184 non-null   float64       
 2   relative_humidity_2m (%)  2184 non-null   int64         
 3   wind_speed_10m (km/h)     2184 non-null   float64       
 4   wind_direction_10m (°)    2184 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 85.4 KB
None


In [89]:
Weather.head(5)

,DateTime,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_direction_10m (°)
0,2017-12-17 00:00:00,-0.9,96,10.4,304
1,2017-12-17 01:00:00,-0.9,96,7.8,292
2,2017-12-17 02:00:00,-2.1,97,10.5,286
3,2017-12-17 03:00:00,-1.9,98,8.8,279
4,2017-12-17 04:00:00,-2.4,99,11.2,274


In [90]:
GiantData.tail(5)

,DateTime,Date,Time,SatMean,SatBand,FlowMean,Latitude,Longitude
8990466,16-03-2018 19:00,16-Mar-2018,19:00,47.2,0-79%,1484,51.631826,-0.095096
8990467,16-03-2018 20:00,16-Mar-2018,20:00,37.4,0-79%,1113,51.631826,-0.095096
8990468,16-03-2018 21:00,16-Mar-2018,21:00,24.2,0-79%,807,51.631826,-0.095096
8990469,16-03-2018 22:00,16-Mar-2018,22:00,18.4,0-79%,622,51.631826,-0.095096
8990470,16-03-2018 23:00,16-Mar-2018,23:00,12.6,0-79%,439,51.631826,-0.095096


In [91]:
# 先转换日期时间格式以匹配
Weather['date'] = pd.to_datetime(Weather['date'], format='%Y-%m-%d %H:%M')
GiantData['DateTime'] = pd.to_datetime(GiantData['DateTime'], format='%d-%m-%Y %H:%M')

# 在合并之前，重命名Weather的日期列，以便与GiantData的对应
Weather.rename(columns={'date': 'DateTime'}, inplace=True)

KeyError: 'date'

In [43]:
# 合并DataFrame，基于DateTime列，选择左外合并以保留GiantData中的所有行
# 注意：这里假设GiantData是左边的DataFrame
Giant_merged = pd.merge(GiantData, Weather[['DateTime', 'temperature_2m (°C)','relative_humidity_2m (%)', 'wind_speed_10m (km/h)','wind_direction_10m (°)']], on='DateTime', how='left')

# 结果是df_merged现在包含了GiantData的所有行和列，加上对应时间点的属性值

In [92]:
Giant_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8990471 entries, 0 to 8990470
Data columns (total 18 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   DateTime                  datetime64[ns]
 1   Date                      object        
 2   Time                      object        
 3   SatMean                   float64       
 4   SatBand                   object        
 5   FlowMean                  int64         
 6   Latitude                  float64       
 7   Longitude                 float64       
 8   temperature_2m (°C)       float64       
 9   relative_humidity_2m (%)  int64         
 10  wind_speed_10m (km/h)     float64       
 11  wind_direction_10m (°)    int64         
 12  SiteType                  int64         
 13  Hour                      int32         
 14  Day                       int32         
 15  Month                     int32         
 16  Year                      int32         
 17  time_ela

In [93]:

Giant_merged['Hour'] = Giant_merged['DateTime'].dt.hour
Giant_merged['Day'] = Giant_merged['DateTime'].dt.day
Giant_merged['Month'] = Giant_merged['DateTime'].dt.month
Giant_merged['Year'] = Giant_merged['DateTime'].dt.year


In [94]:
# set the baseline date and calculate the time elapsed 
# between the baseline date and the date of each row
baseline_date = pd.to_datetime('2017-12-17 00:00')

Giant_merged['time_elapsed'] = (Giant_merged['DateTime'] - baseline_date).dt.total_seconds() / 3600.0

In [96]:
Giant_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8990471 entries, 0 to 8990470
Data columns (total 18 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   DateTime                  datetime64[ns]
 1   Date                      object        
 2   Time                      object        
 3   SatMean                   float64       
 4   SatBand                   object        
 5   FlowMean                  int64         
 6   Latitude                  float64       
 7   Longitude                 float64       
 8   temperature_2m (°C)       float64       
 9   relative_humidity_2m (%)  int64         
 10  wind_speed_10m (km/h)     float64       
 11  wind_direction_10m (°)    int64         
 12  SiteType                  int64         
 13  Hour                      int32         
 14  Day                       int32         
 15  Month                     int32         
 16  Year                      int32         
 17  time_ela

In [97]:
Giant_merged.sample(7)

,DateTime,Date,Time,SatMean,SatBand,FlowMean,Latitude,Longitude,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_direction_10m (°),SiteType,Hour,Day,Month,Year,time_elapsed
949677,2018-02-19 08:00:00,19-Feb-2018,08:00,56.67,0-79%,1399,51.502354,-0.162934,6.4,96,7.1,229,2,8,19,2,2018,1544.0
350749,2018-03-02 06:00:00,2-Mar-2018,06:00,13.00,0-79%,638,51.510500,-0.121418,-0.8,76,33.1,68,2,6,2,3,2018,1806.0
519497,2018-03-16 05:00:00,16-Mar-2018,05:00,2.00,0-79%,72,51.518878,-0.136987,7.2,91,15.9,213,2,5,16,3,2018,2141.0
4923394,2018-01-27 08:00:00,27-Jan-2018,08:00,0.00,0-79%,324,51.597008,-0.015436,3.8,88,16.4,195,2,8,27,1,2018,992.0
2134608,2018-03-08 00:00:00,8-Mar-2018,00:00,42.25,0-79%,3500,51.509951,-0.074792,4.1,91,14.2,187,2,0,8,3,2018,1944.0
6527504,2018-02-07 00:00:00,7-Feb-2018,00:00,11.33,0-79%,307,51.360367,-0.191789,0.4,86,6.1,320,2,0,7,2,2018,1248.0
3639307,2017-12-28 06:00:00,28-Dec-2017,06:00,15.50,0-79%,599,51.470585,-0.124441,-0.9,91,13.4,265,2,6,28,12,2017,270.0


In [49]:
#save another file for backup before dropping the irrelevant columns
Giant_merged.to_csv('Data/CongestionScoot/GiantData_merged1.csv', index=False)


In [2]:
Giant_merged = pd.read_csv('Data/CongestionScoot/GiantData_merged1.csv')

In [3]:
# calculate the number of rows containing missing or NaN values
missing = Giant_merged.isnull().sum()
missing

DateTime                         0
Date                             0
Time                             0
SatMean                          0
SatBand                     546698
FlowMean                         0
Latitude                         0
Longitude                        0
temperature_2m (°C)              0
relative_humidity_2m (%)         0
wind_speed_10m (km/h)            0
wind_direction_10m (°)           0
SiteType                         0
Hour                             0
Day                              0
Month                            0
Year                             0
time_elapsed                     0
dtype: int64

In [4]:

# drop the irrelevant columns
drop_columns = ['Date', 'Time', 'DateTime','SatBand']
Giant_no_species = Giant_merged.drop(drop_columns, axis=1)

In [5]:
# calculate the number of rows containing missing or NaN values
missing = Giant_no_species.isnull().sum()
missing

SatMean                     0
FlowMean                    0
Latitude                    0
Longitude                   0
temperature_2m (°C)         0
relative_humidity_2m (%)    0
wind_speed_10m (km/h)       0
wind_direction_10m (°)      0
SiteType                    0
Hour                        0
Day                         0
Month                       0
Year                        0
time_elapsed                0
dtype: int64

In [6]:
Giant_for_prediction = Giant_no_species.copy()
Giant_for_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8990471 entries, 0 to 8990470
Data columns (total 14 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   SatMean                   float64
 1   FlowMean                  int64  
 2   Latitude                  float64
 3   Longitude                 float64
 4   temperature_2m (°C)       float64
 5   relative_humidity_2m (%)  int64  
 6   wind_speed_10m (km/h)     float64
 7   wind_direction_10m (°)    int64  
 8   SiteType                  int64  
 9   Hour                      int64  
 10  Day                       int64  
 11  Month                     int64  
 12  Year                      int64  
 13  time_elapsed              float64
dtypes: float64(6), int64(8)
memory usage: 960.3 MB


In [7]:
# 检查CUDA设备的可用性
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


In [14]:
device = torch.device("cpu")

In [15]:
# 进行预测
# 导入之前训练好的模型
# 之前定义过的模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        # 定义双向LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_dim * 2, 100) # 由于是双向，所以是hidden_dim的两倍
        self.fc2 = nn.Linear(100, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # 初始化隐藏状态，确保它们也在正确的设备上
        h0 = torch.zeros(self.layer_dim * 2, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim * 2, x.size(0), self.hidden_dim, device=device).requires_grad_()
        
        # 从LSTM层获取输出
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        out = self.dropout(self.relu(self.fc1(out[:, -1, :])))
        out = self.fc2(out)
        return out

input_dim = 14
hidden_dim = 500
layer_dim = 1
output_dim = 1

# 创建模型实例
model_loaded = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)

# 加载保存的状态字典
model_loaded.load_state_dict(torch.load(f"C:/Users/SBH/OneDrive - University College London/#CUSPLondonDataDive2024/2024CUSPLondonDataDive/ModelsResults/LSTM_NO2.pt"))

# 将模型设置为评估模式
model_loaded.eval()

LSTMModel(
  (lstm): LSTM(14, 500, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=1000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [16]:
# change the order of the columns to match the order of the columns in the training data
cols_new_order = [
    'SiteType', 'Latitude', 'Longitude', 'SatMean', 
    'FlowMean', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 
    'wind_speed_10m (km/h)', 'wind_direction_10m (°)', 'time_elapsed', 
    'Hour', 'Day', 'Month', 'Year'
]

In [17]:
y_val_all = pd.DataFrame()
Predict_Data = Giant_for_prediction.astype('float32')
Predict_Data = Predict_Data[cols_new_order]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Predict_Data)
X_val_tensor = torch.tensor(X_scaled).unsqueeze(1).to(device)


In [18]:
# 按批次进行预测 并且将结果保存到y_val_all中
for i in range(100):
    if i!=99:
        begin_idx = i*100000
        end_dix = (i+1)*100000
        with torch.no_grad():
            y_val_pred = model_loaded(X_val_tensor[begin_idx:end_dix]).to('cpu').numpy()
            y_val_pred_df = pd.DataFrame(y_val_pred, columns=['Predicted_NO2'])

        y_val_all = pd.concat([y_val_all, y_val_pred_df], ignore_index=True)
    else:
        begin_idx = i*100000
        with torch.no_grad():
            y_val_pred = model_loaded(X_val_tensor[begin_idx:]).to('cpu').numpy()
            y_val_pred_df = pd.DataFrame(y_val_pred, columns=['Predicted_NO2'])

        y_val_all = pd.concat([y_val_all, y_val_pred_df, ], ignore_index=True)




In [21]:
y_val_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8990471 entries, 0 to 8990470
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Predicted_NO2  float32
dtypes: float32(1)
memory usage: 34.3 MB


In [22]:
Final = Giant_merged.copy()
Final['Pred'] = 0

In [23]:
# 将预测结果存储到Final DataFrame中
for i in range(len(y_val_all)):
    Final['Pred'][i] = y_val_all['Predicted_NO2'][i]

C:\Users\SBH\AppData\Local\Temp\ipykernel_38412\2743735773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final['Pred'][i] = y_val_all['Predicted_NO2'][i]


In [25]:
Final.sample(5)

,DateTime,Date,Time,SatMean,SatBand,FlowMean,Latitude,Longitude,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_direction_10m (°),SiteType,Hour,Day,Month,Year,time_elapsed,Pred
6409301,2018-03-03 21:00:00,3-Mar-2018,21:00,-1.0,NaN,0,51.371995,-0.093631,2.3,93,12.6,110,2,21,3,3,2018,1845.0,28.208279
2892233,2018-01-14 05:00:00,14-Jan-2018,05:00,12.8,0-79%,272,51.448782,-0.041142,1.2,96,6.9,51,2,5,14,1,2018,677.0,25.653717
2461635,2018-01-16 02:00:00,16-Jan-2018,02:00,444.0,>= 100%,2276,51.542892,-0.020257,5.8,84,24.5,244,2,2,16,1,2018,722.0,38.896942
1151274,2018-03-12 14:00:00,12-Mar-2018,14:00,41.0,0-79%,2477,51.556701,-0.146607,10.0,87,14.6,286,2,14,12,3,2018,2054.0,54.590858
5044109,2018-01-17 12:00:00,17-Jan-2018,12:00,49.0,0-79%,1427,51.567741,-0.008730,6.5,65,32.7,262,2,12,17,1,2018,756.0,20.133020


In [26]:
Final.to_csv('Data/FinalPred_NO2.csv', index=False)

In [29]:
simple = Final[['DateTime', 'Pred','Latitude', 'Longitude','SatMean','FlowMean','time_elapsed','Day','Hour']]
simple.to_csv('Data/SimplePred_NO2.csv', index=False)

In [37]:
simple['time_elapsed'].max()

2159.0

### 接下来来plot每一个timestamp的图像并且将其合并转换成GIF图

In [5]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd

In [6]:
# 读取伦敦的空间wards边界数据
london_wards = gpd.read_file('Data/Spatial/London-wards-2018/London-wards-2018_ESRI/London_Ward.shp')

In [7]:
#查看london_wards的CRS
london_wards.crs
# 将london_wards的CRS转换为EPSG:4326
london_wards = london_wards.to_crs(epsg=4326)

In [8]:
simple = pd.read_csv('Data/SimplePred_NO2.csv')
# 将其转换为gdf
Points = gpd.GeoDataFrame(simple, geometry=gpd.points_from_xy(simple.Longitude, simple.Latitude))

In [9]:
# 将Points中'time_elapsed'列的值转换为int格式
Points['time_elapsed'] = Points['time_elapsed'].astype(int)

In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import imageio
import numpy as np

In [11]:
Points['Pred'].min()

-59.08468627929688

In [12]:
Points['Pred'] = Points['Pred'].apply(lambda x: x+59.1)

In [13]:
Points['Pred'].max()

318.11016235351565

In [14]:
# 设置图片保存路径
output_images = []

vmin, vmax = 0, 240  # 例如，根据你的数据调整这些值

for time in range(1790,2160):  # 从0到2159
    # 筛选出当前时间点的数据
    current_points = Points[Points['time_elapsed'] == time]
        # 假设DateTime是字符串格式，我们首先解析这些日期时间
    current_points['DateTime'] = pd.to_datetime(current_points['DateTime'])
    # 提取第一个时间点的年月日时信息作为示例
    if not current_points.empty:
        dt = current_points['DateTime'].iloc[0]  # 仅作为示例
        year, month, day, hour = dt.year, dt.month, dt.day, dt.hour
        date_text = f"Year: {year}   Month: {month}   Day: {day}   Hour: {hour}"
    else:
        date_text = "No data available"

    # 计算每个ward内符合当前时间点的Pred均值
    # 通过空间连接将点分配到相应的ward
    joined = gpd.sjoin(london_wards, current_points, how="left", op="contains")

    # 计算均值
    ward_mean_pred = joined.groupby(joined.index).agg({'Pred': 'mean'}).rename(columns={'Pred': 'mean_pred'})

    # 将均值合并回wards数据集
    london_wards_with_mean = london_wards.join(ward_mean_pred, how="left")


    # 绘制地图
    fig, ax = plt.subplots(figsize=(10, 5))
    london_wards_with_mean.to_crs(epsg=3857).plot(ax=ax, column='mean_pred', cmap='RdYlGn_r', alpha=0.7,
                                                        linewidth=0.5, edgecolor='white',
                                                    vmin=vmin, vmax=vmax,  # 固定颜色映射
                                                    legend=True, legend_kwds={'shrink': 0.7, 'aspect': 10})  # 调整图例尺寸
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    ax.set_axis_off()


    # 调整图例位置
    leg = ax.get_legend()
    if leg:
        leg.set_bbox_to_anchor((0.5, 1))

    # 添加日期时间文字
    plt.figtext(0.5, 0.01, date_text, ha="center", fontsize=12)

    # 保存图片
    img_path = f'Images/TimeStampToGif/ward_map_{time}.png'
    plt.savefig(img_path, dpi=150, bbox_inches='tight')
    plt.close()


c:\Users\SBH\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\SBH\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\SBH\AppData\Local\Temp\ipykernel_29160\1352713168.py:21: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  joined = gpd.sjoin(london_wards,

In [17]:
output_images = []
for i in range(2160):
    img_path = f'Images/TimeStampToGif/ward_map_{i}.png'
    output_images.append(img_path)

In [19]:

# 生成GIF
gif_path = 'Images/wards_animation.gif'
imageio.mimsave(gif_path, [imageio.imread(img) for img in output_images], duration=50)


C:\Users\SBH\AppData\Local\Temp\ipykernel_29160\1320817816.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imageio.mimsave(gif_path, [imageio.imread(img) for img in output_images], duration=50)
